# Comparing MC-Dropout and Moment Propagation

In [1]:
%load_ext autoreload

In [2]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [3]:
BASE_PATH = os.path.join(os.getcwd(), "..", "..")
MODULES_PATH = os.path.join(BASE_PATH, "modules")
DATASET_PATH = os.path.join(BASE_PATH, "datasets")

In [4]:
sys.path.append(MODULES_PATH)

In [5]:
from bayesian import McDropout, MomentPropagation
from models import fchollet_cnn, setup_growth
from data import BenchmarkData, DataSetType

In [6]:
mnist = BenchmarkData(DataSetType.MNIST, os.path.join(DATASET_PATH, "mnist"), dtype=np.float32)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(mnist.inputs, mnist.targets)

In [8]:
# Setup parameters and environment
setup_growth()
num_classes = len(np.unique(mnist.targets))
epochs = 120
batch_size = 80

# Create and fit model
base_model = fchollet_cnn(output=num_classes)
base_model.compile(optimizer="adadelta", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
base_model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

1 Physical GPU's,  1 Logical GPU's
Epoch 1/120
657/657 [==============================] - 11s 17ms/step - loss: 30.2220 - accuracy: 0.1606
Epoch 2/120
657/657 [==============================] - 4s 6ms/step - loss: 10.5834 - accuracy: 0.2941
Epoch 3/120
657/657 [==============================] - 4s 5ms/step - loss: 4.9520 - accuracy: 0.3428
Epoch 4/120
657/657 [==============================] - 4s 5ms/step - loss: 2.9873 - accuracy: 0.3355
Epoch 5/120
657/657 [==============================] - 4s 5ms/step - loss: 2.3417 - accuracy: 0.3448
Epoch 6/120
657/657 [==============================] - 4s 6ms/step - loss: 2.1095 - accuracy: 0.3534
Epoch 7/120
657/657 [==============================] - 4s 6ms/step - loss: 2.0005 - accuracy: 0.3691
Epoch 8/120
657/657 [==============================] - 4s 6ms/step - loss: 1.9008 - accuracy: 0.3969
Epoch 9/120
657/657 [==============================] - 4s 6ms/step - loss: 1.8321 - accuracy: 0.4166
Epoch 10/120
657/657 [==============================

657/657 [==============================] - 4s 6ms/step - loss: 0.6092 - accuracy: 0.8250
Epoch 82/120
657/657 [==============================] - 4s 6ms/step - loss: 0.5965 - accuracy: 0.8298
Epoch 83/120
657/657 [==============================] - 4s 6ms/step - loss: 0.5844 - accuracy: 0.8316
Epoch 84/120
657/657 [==============================] - 4s 5ms/step - loss: 0.5800 - accuracy: 0.8337
Epoch 85/120
657/657 [==============================] - 4s 6ms/step - loss: 0.5826 - accuracy: 0.8352
Epoch 86/120
657/657 [==============================] - 4s 6ms/step - loss: 0.5784 - accuracy: 0.8345
Epoch 87/120
657/657 [==============================] - 4s 6ms/step - loss: 0.5625 - accuracy: 0.8397
Epoch 88/120
657/657 [==============================] - 4s 6ms/step - loss: 0.5585 - accuracy: 0.8408
Epoch 89/120
657/657 [==============================] - 4s 6ms/step - loss: 0.5527 - accuracy: 0.8412
Epoch 90/120
657/657 [==============================] - 4s 6ms/step - loss: 0.5585 - accuracy: 

In [18]:
base_model.evaluate(x_test, y_test)

547/547 [==============================] - 1s 2ms/step - loss: 0.1784 - accuracy: 0.9520


[0.17835696041584015, 0.9520000219345093]

# Compare Entropy values

In [19]:
comp_inputs = x_test[:100]
comp_targets = y_test[:100]

In [29]:
# McDropout model
mc_model = McDropout(base_model)
mc_pred = mc_model(comp_inputs, sample_size=500)
print("Pred. shape: {} '(num_datapoints, sample_size, target_size)'".format(mc_pred.shape))

Pred. shape: (100, 500, 10) '(num_datapoints, sample_size, target_size)'


In [30]:
mc_nll = mc_model.nll(mc_pred, comp_targets)

In [31]:
np.sum(mc_nll)

16.620937199507757

In [23]:
# Moment propagation model
mp_model = MomentPropagation(base_model)
mp_exp, mp_var = mp_model(comp_inputs)

In [24]:
mp_nll = mp_model.nll(mp_exp, comp_targets)

In [25]:
np.sum(mp_nll)

4.746258348154156